In [2]:
source("../funcs/limma.R")
source("../funcs/funcsR.R")
suppressMessages(library(fgsea))
suppressMessages(library('ggplot2'))
suppressMessages(library('ggpubr'))
suppressMessages(library('ggdendro'))
suppressMessages(library(RColorBrewer))
suppressMessages(library(EnhancedVolcano))
suppressMessages(library(xCell))

---
# Analysis of SMM Transcriptomes pt. 2

This notebooks contains R code & scripts to run differential exression, fGSEA, and create plots.

**Author**: [Shankara Anand](sanand@broadinstitute.org)

---

## 0. Load Data

In [3]:
# Load TPM
tpm.df <- read.table("supplement/table9a_janssen_tpm.tsv",sep='\t',header=T,row.names=1)

# Load Norm TPM
tpm.norm.df <- read.table("supplement/table9b_janssen_tpm_norm.tsv",sep='\t', header=T, row.names=1)
tpm.norm.df <- data.frame(t(tpm.norm.df))

# Load counts
counts.df <- read.table("supplement/table9c_janssen_counts.tsv",sep='\t',header=T,row.names=1)

# Genes 
gene.df <- read.table("supplement/table9d_gene_name.tsv",sep='\t',header=T,row.names=1)
gene.df <- gene.df[gene.df$biotype=='protein_coding',]
full.gene.df <- read.table("supplement/table9d_gene_name.tsv",sep='\t',header=T,row.names=1)
genes.to.use <- intersect(rownames(gene.df),rownames(tpm.df))

# Labels
labels.df <- read.table("../Fig1/supplement/table4_sample_cluster_id.tsv",sep='\t',header=T,row.names=1)
samples.to.use <- intersect(colnames(counts.df),rownames(labels.df))

In [4]:
labels.df <- labels.df[samples.to.use,]
counts.df <- counts.df[genes.to.use,samples.to.use]
gene.df <- gene.df[genes.to.use,"Description", drop=F]

In [5]:
tpm.norm.df <- tpm.norm.df[samples.to.use,]
tpm.norm.df$consensus_nmf <- as.factor(as.character(labels.df[samples.to.use,]$consensus_nmf))

In [6]:
X.df <- read.table("../Fig1/supplement/table3_smm_filt_matrix.tsv",sep='\t',header=T,row.names=1)
X.df <- X.df[samples.to.use,]

In [7]:
addName <- function(s){
    if(s=="C1"){
        return("HNT")
    }else if(s=='C2'){
        return("HMC")
    }else if(s=='C3'){
        return("FMD")
    }else if(s=='C4'){
        return("HKR")
    }else if(s=='C5'){
        return("CND")
    }else if(s=='C6'){
        return("HNF")
    }
}

In [8]:
tpm.norm.df$id <- factor(sapply(tpm.norm.df$consensus_nmf, addName), levels = c("HNT", "HMC", "FMD",  "HKR", "CND", "HNF"))

---

## 1. Expression Comparisons

* Barplots showing expression of genes across clusters

In [48]:
# plotTPM(tpm.norm.df, "BCL2L1", list(  c("C1","C2")), full.gene.df, x='id', label.y=3, label.pos='none')
# plotTPM(tpm.norm.df, "BCL2", list(  c("C1|HNT","C2|HMC")), full.gene.df, x='id', label.y=3, label.pos='none')

In [49]:
tpm.norm.df$Hyperdiploidy<- ifelse(tpm.norm.df$consensus_nmf %in% c("C3","C5"), "- HP", "+ HP")

In [50]:
pdf('figures/fig2e_myc_hp.pdf', width=4, height=4)
plotTPM(tpm.norm.df, "MYC", c("- HP", "+ HP"), full.gene.df, x='Hyperdiploidy', label.x=.75, label.y=1, w=4, h=4, ylim.min=1, label.pos="none", palette=NULL)
dev.off()

Warning message:
“Computation failed in `stat_signif()`:
not enough 'y' observations”

png 
  2

In [51]:
tpm.norm.df$cnd <- factor(ifelse(tpm.norm.df$consensus_nmf=="C5", "CND", "Rest"), levels=c("Rest","CND"))
p1 <- plotTPM(tpm.norm.df, "CCND1", c("Rest", "CND"), full.gene.df, x='cnd', label.x=.75, label.y=0, w=4, h=4, label.pos="none", palette=NULL)

tpm.norm.df$fmd <- factor(ifelse(tpm.norm.df$consensus_nmf %in% c("C3","C2"), "FMD/HMC", "Rest"), levels=c("Rest","FMD/HMC"))
p2 <- plotTPM(tpm.norm.df, "CCND2", c("Rest", "FMD"), full.gene.df, x='fmd', label.x=.75, label.y=0, w=4, h=4, label.pos="none", palette=NULL)

tpm.norm.df$hnt <- factor(ifelse(tpm.norm.df$consensus_nmf=="C1", "HNT", "Rest"), levels=c("Rest","HNT"))
p3 <- plotTPM(tpm.norm.df, "MCL1", c("Rest", "HNT"), full.gene.df, x='hnt', label.x=.75, label.y=7, w=4, h=4, ylim.min=7, label.pos="none", palette=NULL)

pdf('figures/figS4cde_tpm_barplots.pdf', width=4, height=12)
ggarrange(p1,p2,p3,labels = c("C", "D", "E"), ncol = 1, nrow = 3)
dev.off()

Warning message:
“Computation failed in `stat_signif()`:
not enough 'y' observations”Warning message:
“Computation failed in `stat_signif()`:
not enough 'y' observations”Warning message:
“Computation failed in `stat_signif()`:
not enough 'y' observations”

png 
  2

In [52]:
p1 <- plotTPM(tpm.norm.df, "MCL1", list(  c("HNT","FMD"), c("HMC","FMD"), c("FMD","CND")), full.gene.df, x="id", label.x=.9, ylim.min=6, label.y=6, label.pos='none')
p2 <- plotTPM(tpm.norm.df, "MYC", NA, full.gene.df, x="id", label.x=.9, label.pos='none') + scale_y_continuous(breaks=c(0,2,4,6,8,10),limits = c(0, NA))
p3 <- plotTPM(tpm.norm.df, "CCND1", NA, full.gene.df, x="id", label.x=.9, label.pos='none') + scale_y_continuous(breaks=c(0,2,4,6,8,10),limits = c(0, NA))
p4 <- plotTPM(tpm.norm.df, "CCND2", NA, full.gene.df, x="id", label.x=.9, label.pos='none') + scale_y_continuous(breaks=c(0,2,4,6,8,10),limits = c(0, NA))

pdf('figures/fig2abcd_tpm_barplots.pdf', width=8, height=20)
ggarrange(p1,p2,p3,p4,labels = c("A", "B", "C", "D"), ncol = 1, nrow = 4)
dev.off()

Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.
Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.
Scale for 'y' is already present. Adding another scale for 'y', which will
replace the existing scale.
Warning message:
“Computation failed in `stat_signif()`:
missing value where TRUE/FALSE needed”Warning message:
“Computation failed in `stat_signif()`:
missing value where TRUE/FALSE needed”Warning message:
“Computation failed in `stat_signif()`:
missing value where TRUE/FALSE needed”

png 
  2

---

## 2. Differential Expression One vs. Rest (Clusters)

### One vs. Rest

In [16]:
# For all samples with matching RNA
DE.DIR <- "supplement/diffexp"
dir.create(DE.DIR)

for (s in unique(labels.df$consensus_nmf)){
    labels.df$de <- labels.df$consensus_nmf==s
    
    # Run Differential Expression
    de.df <- RunDiffExprAnalysisLimma(
        counts.df, 
        labels.df[,c('de'), drop=FALSE], 
        genes.df=gene.df
    )
    
    write.table(de.df, file.path(DE.DIR, paste("cluster_", s ,".tsv", sep='')), quote=FALSE, sep="\t")
}

# Only within HP clusters
DE.DIR <- "supplement/diffexp_hp"
dir.create(DE.DIR)

labels.hp.df <- labels.df[labels.df$consensus_nmf %in% c("C1","C2","C4","C6"),]
counts.hp.df <- counts.df[,rownames(labels.hp.df)]

for (s in unique(labels.hp.df$consensus_nmf)){
    labels.hp.df$de <- labels.hp.df$consensus_nmf==s
    
    # Run Differential Expression
    de.df <- RunDiffExprAnalysisLimma(
        counts.hp.df, 
        labels.hp.df[,c('de'), drop=FALSE], 
        genes.df=gene.df
    )
    
    write.table(de.df, file.path(DE.DIR, paste("cluster_subset_HP_", s ,".tsv", sep='')), quote=FALSE, sep="\t")
}

Warning message in dir.create(DE.DIR):
“'supplement/diffexp' already exists”

  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 67
  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 0
  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 81
  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 27
  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 7
  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 4


Warning message in dir.create(DE.DIR):
“'supplement/diffexp_hp' already exists”

  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 0
  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 0
  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 0
  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 7


In [17]:
c4.de.hl <- read.table("../data/ref/de_highlights/C4_selected_DE_genes.txt",sep='\t',header=T)
c3.de.hl <- read.table("../data/ref/de_highlights/C3_selected_DE_genes.txt",sep='\t',header=T)

In [18]:
pdf('figures/fig2f_volcano_clust_FMD_de.pdf', width=8, height=10)
de.df <- read.table("supplement/diffexp/cluster_C3.tsv",sep='\t')
plotVolcano(
    de.df, 
    selectLab=append(as.vector(c3.de.hl$upregulated_gene_name),as.vector(c3.de.hl$downregulated_gene_name)),
    widthConnectors = 0.25,
    drawConnectors=T, 
    boxedLabels = TRUE,
    h=10,
    w=8,
    labSize=2,
    labvjust=1,
    labhjust=1,
    title='FMD'
)
dev.off()

pdf('figures/fig2g_volcano_clust_CND_de.pdf', width=8, height=10)
de.df <- read.table("supplement/diffexp/cluster_C5.tsv",sep='\t')
plotVolcano(
    de.df, 
    selectLab=append(as.vector(c4.de.hl$Upregulated_gene_name),as.vector(c4.de.hl$downregulated_gene_name)),
    widthConnectors = 0.25,
    drawConnectors=T, 
    boxedLabels = TRUE,
    h=10,
    w=8,
    labSize=2,
    labvjust=0.5,
    labhjust=1,
    title='CND'
)
dev.off()

png 
  2

png 
  2

In [19]:
# #pdf('figures/volcano_clust_C1_de.pdf', width=8, height=10)
# de.df <- read.table("supplement/diffexp/cluster_C1.tsv",sep='\t')
# plotVolcano(
#     de.df,
#     widthConnectors = 0.25,
#     drawConnectors=T, 
#     boxedLabels = TRUE,
#     h=10,
#     w=8,
#     labSize=2,
#     labvjust=1,
#     labhjust=1,
#     title='C1'
# )
# #dev.off()

# #pdf('figures/volcano_clust_C2_de.pdf', width=8, height=10)
# de.df <- read.table("supplement/diffexp/cluster_C2.tsv",sep='\t')
# plotVolcano(
#     de.df,
#     widthConnectors = 0.25,
#     drawConnectors=T, 
#     boxedLabels = TRUE,
#     h=10,
#     w=8,
#     labSize=2,
#     labvjust=1,
#     labhjust=1,
#     title='C2'
# )
# #dev.off()

# #pdf('figures/volcano_clust_C6_de.pdf', width=8, height=10)
# de.df <- read.table("supplement/diffexp/cluster_C6.tsv",sep='\t')
# plotVolcano(
#     de.df,
#     widthConnectors = 0.25,
#     drawConnectors=T, 
#     boxedLabels = TRUE,
#     h=10,
#     w=8,
#     labSize=2,
#     labvjust=1,
#     labhjust=1,
#     title='C6'
# )
# #dev.off()

---

## 2. Misc. Differential Expression Comparisons

In [20]:
DE.DIR <- "supplement/diffexp_misc"
dir.create(DE.DIR)

Warning message in dir.create(DE.DIR):
“'supplement/diffexp_misc' already exists”

### HP vs Rest

In [21]:
labels.df$de <- labels.df$consensus_nmf %in% c("C1","C2","C4","C6")

# Run Differential Expression
de.df <- RunDiffExprAnalysisLimma(counts.df, labels.df[,c('de'), drop=FALSE], genes.df=gene.df)
write.table(de.df, file.path(DE.DIR, "cluster_hp_vs_rest.tsv"), quote=FALSE, sep="\t")

de.df[de.df$gene_name =='MYC',]

  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 740


,logFC,AveExpr,t,P.Value,adj.P.Val,B,gene_name
ENSG00000136997.10,0.9796356,6.950225,3.256196,0.001604338,0.03984064,-1.544484,MYC


### 1q Clusters

In [22]:
labels.1q.df <- labels.df[labels.df$consensus_nmf %in% c("C3","C6"),]
labels.1q.df$de <- labels.1q.df$consensus_nmf=="C3"

# Run Differential Expression
de.df <- RunDiffExprAnalysisLimma(counts.df[,rownames(labels.1q.df)], labels.1q.df[,c('de'), drop=FALSE], genes.df=gene.df)
write.table(de.df, file.path(DE.DIR, "cluster_subset_1q_vs_rest.tsv"), quote=FALSE, sep="\t")

  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 0


### t(11;14)

In [51]:
labels.df$t1114 <- as.logical(X.df[,'t.11.14.'])

# Run Differential Expression
de.df <- RunDiffExprAnalysisLimma(counts.df[,rownames(labels.df)], labels.df[,c('t1114'), drop=FALSE], genes.df=gene.df,)
write.table(de.df, file.path(DE.DIR, "t1114_vs_rest.tsv"), quote=FALSE, sep="\t")

  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 114


### 1q Amp vs Rest

In [52]:
labels.df$one_q <- as.logical(X.df[rownames(labels.df),'X1q_gain'])

# Run Differential Expression
de.df <- RunDiffExprAnalysisLimma(counts.df, labels.df[,c('one_q'), drop=FALSE], genes.df=gene.df)
write.table(de.df, file.path(DE.DIR, "1q_vs_rest.tsv"), quote=FALSE, sep="\t")

  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 253


### 1p Del vs. Rest

In [55]:
labels.df$one_p <- as.logical(X.df[rownames(labels.df),'X1p_del'])

# Run Differential Expression
de.df <- RunDiffExprAnalysisLimma(counts.df, labels.df[,c('one_p'), drop=FALSE], genes.df=gene.df)
write.table(de.df, file.path(DE.DIR, "1p_del_vs_rest.tsv"), quote=FALSE, sep="\t")

  * model matrix dimensions: 2
Differentially expressed genes at 0.05 FDR: 3


---

## 3. Pathways

* Run fGSEA for differential expression results
* Rank by logFC, logFC * -log10(adj.p-val)

In [23]:
GMTS = c(
    "../data/ref/gmts/h.all.v7.0.symbols.gmt.txt",
    "../data/ref/gmts/c2.cp.kegg.v7.0.symbols.gmt.txt",
    "../data/ref/gmts/mm_sigs_custom.gmt.txt",
    "../data/ref/gmts/staudt_2020.gmt.txt"
)

GMT_list <- c(gmtPathways(GMTS[1]),gmtPathways(GMTS[2]),gmtPathways(GMTS[3]))
SEED <- 42

In [24]:
DE.DIR <- "supplement/diffexp"

de.df <- list()
c <- 1

for (file in list.files("supplement/diffexp")){
    file.name <- file.path(DE.DIR, file)
    de.df[[c]] <- read.table(file.name, sep='\t', header=T, row.names=1)
    de.df[[c]]$id <- strsplit(file,'\\.')[[1]][1]

    c = c+1
}

de.df <- do.call("rbind", de.df)
de.df$id <- gsub("cluster_", "", de.df$id)

In [25]:
# Full Pathway Enrichments
e.df <- runAllGSEA(de.df, GMT_list, seed=SEED)
write.table(e.df, "supplement/table11a_enrich_v1.tsv",sep='\t', quote = T)

# Full Pathway Enrichments for Gene-Sets in Staudt et al
e.staudt.df <- runAllGSEA(de.df, gmtPathways(GMTS[4]), seed=SEED)
write.table(e.staudt.df, "supplement/table11b_enrich_staudt.tsv",sep='\t', quote = T)

# Stricter Pathway Enrichments (ranked by padj*signed FC)
e2.df <- runAllGSEA(de.df, GMT_list, how='padj', seed=SEED)
write.table(e2.df, "supplement/table11c_enrich_v2.tsv",sep='\t', quote = T)

# Stricter Pathway Enrichments (ranked by padj*signed FC) for Gene-Sets in Staudt et al
e.staudt.2.df <- runAllGSEA(de.df, gmtPathways(GMTS[4]), how='padj', seed=SEED)
write.table(e.staudt.2.df, "supplement/table11d_enrich_staudt_v2.tsv",sep='\t', quote = T)

Warning message in fgsea(GMTS, R, 10000, minSize = 3):
“There are duplicate gene names, fgsea may produce unexpected results”Warning message in fgsea(GMTS, R, 10000, minSize = 3):
“There are duplicate gene names, fgsea may produce unexpected results”Warning message in fgsea(GMTS, R, 10000, minSize = 3):
“There are duplicate gene names, fgsea may produce unexpected results”Warning message in fgsea(GMTS, R, 10000, minSize = 3):
“There are duplicate gene names, fgsea may produce unexpected results”Warning message in fgsea(GMTS, R, 10000, minSize = 3):
“There are duplicate gene names, fgsea may produce unexpected results”Warning message in fgsea(GMTS, R, 10000, minSize = 3):
“There are duplicate gene names, fgsea may produce unexpected results”Warning message in fgsea(GMTS, R, 10000, minSize = 3):
“There are duplicate gene names, fgsea may produce unexpected results”Warning message in fgsea(GMTS, R, 10000, minSize = 3):
“There were 1 pathways for which P-values were not calculated properly

### Fig 2C

In [26]:
hm.to.remove <- c(
    "HALLMARK_MYOGENESIS",
    "HALLMARK_KRAS_SIGNALING_UP",
    "HALLMARK_KRAS_SIGNALING_DN",
    "HALLMARK_ESTROGEN_RESPONSE_LATE",
    "HALLMARK_ESTROGEN_RESPONSE_EARLY",
    "HALLMARK_XENOBIOTIC_METABOLISM",
    "HALLMARK_ANGIOGENESIS",
    "HALLMARK_APICAL_JUNCTION",
    "HALLMARK_COAGULATION",
    "HALLMARK_UV_RESPONSE_DN",
    "HALLMARK_ALLOGRAFT_REJECTION",
    "HALLMARK_G2M_CHECKPOINT",
    "HALLMARK_SPERMATOGENESIS",
    "HALLMARK_UV_RESPONSE_UP"
)

kegg.to.keep <- c(
    "KEGG_CYTOSOLIC_DNA_SENSING_PATHWAY",
    "KEGG_TGF_BETA_SIGNALING_PATHWAY",
    "KEGG_JAK_STAT_SIGNALING_PATHWAY",
    "KEGG_RIBOSOME"
)

staudt.plot.pathways <- c(
    "B_cell_memory_Newman",
    "Blimp_proliferaton_repressed",
    "Blood_Modules-3.3_Inflammation-2",
    "Blood_Module−1.5_Myeloid_lineage−1",
    "MCL_proliferation_survival",
    "Dendritic_cell_CD123pos_blood",
    "Dendritic_cell_activated_Newman",
    "Macrophage_M1_Newman",
    "Macrophage_M2_Newman",
    "Monocyte_Newman",
    "Cell_cycle_Liu",
    "Regulatory_T cell_Newman",
    "CD4_T_cell_memory_activated_Newman",
    "CD8_T_cell_Newman",
    "NK_cell_resting_Newman",
    "NK_cell_activated_Newman",
    "Gamma_delta_T cell_Newman",
    "CD8_T_effectorUp_memoryIm_NaiveDn",
    "XBP1_target_secretory"
)

#### Fig2C Balloon Plot with fGSEA Ranked by LogFC

In [40]:
# Load Enrichment
e.df <- read.table("supplement/table11a_enrich_v1.tsv")
e.stuadt.join.df <- read.table("supplement/table11b_enrich_staudt.tsv",sep='\t',header=T)

# Rename kegg, hallmark, & myeloma signatures
kegg.e.df <- e.df[grepl("^KEGG", e.df$pathway),]
kegg.e.df <- kegg.e.df[kegg.e.df$pathway %in% kegg.to.keep,]

hm.e.df <- e.df[grepl("^HALLMARK", e.df$pathway),]
hm.e.df <- hm.e.df[!hm.e.df$pathway %in% hm.to.remove,]

mm.e.df <- e.df[grepl("^MYELOMA", e.df$pathway),]
mm.e.df <- mm.e.df[grepl("UP$", mm.e.df$pathway),]

# Join Sataudt Signatures
e.stuadt.filt.join.df <- e.stuadt.join.df[e.stuadt.join.df$pathway %in% staudt.plot.pathways,]

# Group Names
kegg.e.df$grouping <- "Cellular Pathways"
hm.e.df$grouping <- "Cellular Pathways"
mm.e.df$grouping <- "Myeloma"
e.stuadt.filt.join.df[grepl("Newman$", e.stuadt.filt.join.df$pathway),"grouping"] <- "Immune"
e.stuadt.filt.join.df[!grepl("Newman$", e.stuadt.filt.join.df$pathway),"grouping"] <- "Cellular Pathways"

e.stuadt.filt.join.df[e.stuadt.filt.join.df$pathway=="Dendritic_cell_CD123pos_blood","grouping"] <- "Immune"
e.stuadt.filt.join.df[e.stuadt.filt.join.df$pathway=="CD8_T_effectorUp_memoryIm_NaiveDn","grouping"] <- "Immune"

# Intersect for Common Columns
common_cols <- intersect(colnames(kegg.e.df), colnames(e.stuadt.filt.join.df))

full.e.df <- rbind(
  kegg.e.df[,common_cols],
  hm.e.df[,common_cols],
  mm.e.df[,common_cols],
  e.stuadt.filt.join.df[,common_cols]
)

full.e.df$grouping <- as.factor(full.e.df$grouping)
full.e.df$id <- as.factor(full.e.df$id)

# Merge Renamed Pathways
rename.df <- read.table('../data/ref/rename_pathways.tsv',sep='\t',header=T)
x <- merge(full.e.df, rename.df[,c('pathway','new_pathway_name')])
x$pathway <- x$new_pathway_name
x$id <- factor(sapply(x$id, addName), levels = c("HNT", "HMC", "FMD", "HKR", "CND", "HNF"))

In [43]:
pdf("figures/fig2h_balloon_figure.pdf", width=7, height=17) 
plotGSEA_v2(x, h=18, w=8)
dev.off()

png 
  2

#### Fig2C Balloon Plot with fGSEA Ranked by LogFC * -log10(adj. pval)

In [33]:
# Load Enrichment
e.df <- read.table("supplement/table11c_enrich_v2.tsv")
e.stuadt.join.df <- read.table("supplement/table11d_enrich_staudt_v2.tsv",sep='\t',header=T)

# Rename kegg, hallmark, & myeloma signatures
kegg.e.df <- e.df[grepl("^KEGG", e.df$pathway),]
kegg.e.df <- kegg.e.df[kegg.e.df$pathway %in% kegg.to.keep,]

hm.e.df <- e.df[grepl("^HALLMARK", e.df$pathway),]
hm.e.df <- hm.e.df[!hm.e.df$pathway %in% hm.to.remove,]

mm.e.df <- e.df[grepl("^MYELOMA", e.df$pathway),]
mm.e.df <- mm.e.df[grepl("UP$", mm.e.df$pathway),]

# Join Sataudt Signatures
e.stuadt.filt.join.df <- e.stuadt.join.df[e.stuadt.join.df$pathway %in% staudt.plot.pathways,]

# Group Names
kegg.e.df$grouping <- "Cellular Pathways"
hm.e.df$grouping <- "Cellular Pathways"
mm.e.df$grouping <- "Myeloma"
e.stuadt.filt.join.df[grepl("Newman$", e.stuadt.filt.join.df$pathway),"grouping"] <- "Immune"
e.stuadt.filt.join.df[!grepl("Newman$", e.stuadt.filt.join.df$pathway),"grouping"] <- "Cellular Pathways"

e.stuadt.filt.join.df[e.stuadt.filt.join.df$pathway=="Dendritic_cell_CD123pos_blood","grouping"] <- "Immune"
e.stuadt.filt.join.df[e.stuadt.filt.join.df$pathway=="CD8_T_effectorUp_memoryIm_NaiveDn","grouping"] <- "Immune"

# Intersect for Common Columns
common_cols <- intersect(colnames(kegg.e.df), colnames(e.stuadt.filt.join.df))

full.e.df <- rbind(
  kegg.e.df[,common_cols],
  hm.e.df[,common_cols],
  mm.e.df[,common_cols],
  e.stuadt.filt.join.df[,common_cols]
)

full.e.df$grouping <- as.factor(full.e.df$grouping)
full.e.df$id <- as.factor(full.e.df$id)

# Merge Renamed Pathways
rename.df <- read.table('../data/ref/rename_pathways.tsv',sep='\t',header=T)
x <- merge(full.e.df, rename.df[,c('pathway','new_pathway_name')])
x$pathway <- x$new_pathway_name
x$id <- factor(sapply(x$id, addName), levels = c("HNT", "HMC", "FMD", "HKR", "CND", "HNF"))

In [34]:
pdf("figures/figS4a_balloon_figure_v2.pdf", width=7, height=17) 
plotGSEA_v2(x, h=18, w=8)
dev.off()

png 
  2

#### Supplemental plots of all other pathways

In [35]:
e.df <- read.table("supplement/table11a_enrich_v1.tsv")
e.df$id <- factor(sapply(e.df$id, addName), levels = c("HNT", "HMC", "FMD", "HKR", "CND", "HNF"))

In [36]:
pdf("figures/figS5a_balloon_hallmark.pdf", width=8, height=10) 
plotGSEA(e.df, filter='HALLMARK', w=10, h=10, fix_id=F)
dev.off()

pdf("figures/figS5b_balloon_kegg.pdf", width=10, height=18) 
plotGSEA(e.df, filter='KEGG', w=10, h=20, fix_id=F)
dev.off()

e.df.mm <- dplyr::filter(e.df, grepl("MYELOMA", pathway))

pdf("figures/figS5c_balloon_myeloma_up.pdf", width=8, height=5) 
plotGSEA(e.df.mm, filter='UP', w=8, h=5, fix_id=F)
dev.off()

pdf("figures/figS5d_balloon_myeloma_down.pdf", width=8, height=5) 
plotGSEA(e.df.mm, filter='DOWN', w=8, h=5, fix_id=F)
dev.off()

png 
  2

png 
  2

png 
  2

png 
  2

---
## 4. Myeloma Signatures

* Mean expression of genes in Sonnevald + Zhan

In [16]:
sigs.full.df <- read.table("supplement/table10c_combined_mean_sigs.tsv",sep='\t',header=T,row.names=1)
sigs.full.df$consensus_nmf <- factor(sapply(sigs.full.df$consensus_nmf, addName), levels = c("HNT", "HMC", "FMD", "HKR", "CND", "HNF"))

In [19]:
p1 <- plotSig(sigs.full.df, "CD.2", list(  c("HNT","CND"), c("HMC","CND"), c("FMD","CND"), c("HKR","CND"), c("HNF","CND")), label.y=1, label.x=4.5, ylim.min=1, label.pos='none')
p2 <- plotSig(sigs.full.df, "CD.1", list(  c("HNT","CND"), c("HMC","CND"), c("FMD","CND"), c("HKR","CND"), c("HNF","CND")), label.y=1, label.x=4.5, ylim.min=1, label.pos='none')
p3 <- plotSig(sigs.full.df, "LB", list(c("FMD","CND"), c("CND","HNF"),c("HKR","HNF")), label.y=1, label.x=4.5, ylim.min=1, label.pos='none')
p4 <- plotSig(sigs.full.df, "HP", list( c("HNT","HMC"), c("HNT","FMD"), c("HNT","CND"), c("HNT","HKR"), c("HNT","HNF")), label.x=4.5, label.y=1.5, ylim.min=1.5, label.pos='none')
p5 <- plotSig(sigs.full.df, "MS", list(  c("HNT","FMD"), c("HMC","FMD"),   c("FMD","HNF"),  c("FMD","HKR"), c("FMD","CND")), label.x=4.5, label.y=0, label.pos='none')
p6 <- plotSig(sigs.full.df, "PRL3", list(  c("HNT","FMD"), c("HMC","FMD"),   c("CND","HNF"),  c("CND","HKR"), c("FMD","CND")), label.x=4.5, label.y=0.75, ylim.min=.75, label.pos='none')

pdf('figures/figS4b_myeloma_mean_sigs.pdf', width=8, height=30)
ggarrange(p1,p2,p3,p4,p5,p6,labels = c("A", "B", "C", "D", "E", "F"), ncol = 1, nrow = 6)
dev.off()

png 
  2